In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm

In [5]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi

from exact_sync.v1.models import ImageSet, Product, AnnotationType, Image, Annotation
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

## Connect to EXACT

In [6]:
configuration = Configuration()
configuration.username = 'exact' #'exact'
configuration.password = 'exact' #'exact'
configuration.host = "http://127.0.0.1:1337" #"http://127.0.0.1:8000"

client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

In [7]:
annotation_types_points = {}

for product in product_api.list_products(name="Reference_Points").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id).results:
        annotation_types_points[annotation_type.id] = annotation_type

annotation_types_points

{21: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 21,
  'name': 'L0',
  'node_count': 0,
  'product': 7,
  'sort_order': 0,
  'vector_type': 1},
 22: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 22,
  'name': 'L1',
  'node_count': 0,
  'product': 7,
  'sort_order': 0,
  'vector_type': 1},
 23: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 23,
  'name': 'L2',
  'node_count': 0,
  'product': 7,
  'sort_order': 0,
  'vector_type': 1},
 24: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': False,
  'e

In [8]:
annotations = []

for image_set_name in tqdm(["Single_HE", "Multi_HE", "Single_IHC", "Multi_IHC"]):
    
    scanner = image_set_name.split("_")[1]
    image_set = image_sets_api.list_image_sets(name=image_set_name, expand="product_set,product_set.annotationtype_set").results[0]

    for image_id in image_set.images:

        image = images_api.retrieve_image(id=image_id)
        
        image_width, image_height = image.width, image.height

        for anno in annotations_api.list_annotations(image=image_id, pagination=False, # BoundBox ID
                                                     deleted=False, fields="annotation_type,id,image,vector,unique_identifier").results:

            image_type = "MSSC" if "Multi" in image_set_name else "RSC"
            
            if anno.annotation_type in annotation_types_points:
            
                anno.annotation_type = annotation_types_points[anno.annotation_type]
                annotations.append([scanner, image.id, image.name, image_type, image_width, image_height, 
                                    anno.id, anno.vector, anno.unique_identifier, anno.annotation_type.id, 
                                    anno.annotation_type.name])


annotations = pd.DataFrame(annotations, columns=["scanner", "image_id", "image_name", "image_type", "image_width", "image_height",
                                                 "id", "vector", "unique_identifier", "annotation_type", "type_name"])
annotations.head()

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:14<00:00,  3.54s/it]


,scanner,image_id,image_name,image_type,image_width,image_height,id,vector,unique_identifier,annotation_type,type_name
0,HE,325,CRC-AI-15 40X.tif,RSC,86256,86432,26106,"{'x1': 4236, 'x2': 4434, 'y1': 8043, 'y2': 8241}",f52b1e31-3df8-4041-9902-66bc6edef761,21,L0
1,HE,325,CRC-AI-15 40X.tif,RSC,86256,86432,26107,"{'x1': 6352, 'x2': 6552, 'y1': 23748, 'y2': 23...",b8ca1cea-38ef-4818-b050-d77ef0e44e21,22,L1
2,HE,325,CRC-AI-15 40X.tif,RSC,86256,86432,26108,"{'x1': 6158, 'x2': 6358, 'y1': 38586, 'y2': 38...",9d817eda-dd12-49b4-a942-9d7ddd77f305,23,L2
3,HE,325,CRC-AI-15 40X.tif,RSC,86256,86432,26109,"{'x1': 11860, 'x2': 12060, 'y1': 52464, 'y2': ...",04454db4-2e29-4e18-8f8e-271d8e154507,24,L3
4,HE,325,CRC-AI-15 40X.tif,RSC,86256,86432,26110,"{'x1': 6827, 'x2': 7025, 'y1': 68732, 'y2': 68...",81ee0679-7857-4fc4-a402-df65b0576831,25,L4


In [9]:
annotations.to_csv("GT_D240.csv", index=False)